In [1]:
import gc
import os
import sys
import uproot
import pandas
import numpy as np
import keras
import tensorflow as tf
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Dropout, AlphaDropout, Activation, BatchNormalization
from keras.callbacks import ModelCheckpoint, CSVLogger
from tqdm import tqdm_notebook
from keras_tqdm import TQDMNotebookCallback
import matplotlib.pyplot as plt
import functools

sys.path.insert(0, "../python")
from common import *
from WeightManager import WeightManager

C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
print(len(input_branches))
input_branches

181


['pt',
 'eta',
 'mass',
 'decayMode',
 'dxy',
 'dxy_sig',
 'dz',
 'ip3d',
 'ip3d_sig',
 'hasSecondaryVertex',
 'flightLength_r',
 'flightLength_dEta',
 'flightLength_dPhi',
 'flightLength_sig',
 'chargedIsoPtSum',
 'chargedIsoPtSumdR03',
 'footprintCorrection',
 'footprintCorrectiondR03',
 'neutralIsoPtSum',
 'neutralIsoPtSumdR03',
 'neutralIsoPtSumWeight',
 'neutralIsoPtSumWeightdR03',
 'photonPtSumOutsideSignalCone',
 'photonPtSumOutsideSignalConedR03',
 'puCorrPtSum',
 'pt_weighted_deta_strip',
 'pt_weighted_dphi_strip',
 'pt_weighted_dr_signal',
 'pt_weighted_dr_iso',
 'leadingTrackNormChi2',
 'e_ratio',
 'gj_angle_diff',
 'n_photons',
 'emFraction',
 'inside_ecal_crack',
 'has_gsf_track',
 'gsf_ele_matched',
 'gsf_ele_pt',
 'gsf_ele_dEta',
 'gsf_ele_dPhi',
 'gsf_ele_energy',
 'gsf_ele_Ee',
 'gsf_ele_Egamma',
 'gsf_ele_Pin',
 'gsf_ele_Pout',
 'gsf_ele_Eecal',
 'gsf_ele_dEta_SeedClusterTrackAtCalo',
 'gsf_ele_dPhi_SeedClusterTrackAtCalo',
 'gsf_ele_mvaIn_sigmaEtaEta',
 'gsf_ele_mvaI

In [3]:
input_file = 'file://C:/Users/konst/workspace/tau-ml/tuples/training_v4/all_shuffled.root'
weight_file = 'C:/Users/konst/workspace/tau-ml/tuples/training_v4/all_shuffled_weights.h5'

In [4]:
X, Y = ReadXY(input_file, 'taus')#, entrystop=int(7e5))
X.shape, Y.shape

((65452708, 181), (65452708, 4))

In [7]:
#weightManager = WeightManager(weight_file, calc_weights=False)
weightManager = WeightManager('20L1024N50D_relu_0723_s1_last_pass_weights.hdf5', calc_weights=False, first_block=False)

In [8]:
TauLosses.Le_sf = 1
TauLosses.Lmu_sf = 2
TauLosses.Ljet_sf = 5

In [9]:
def create_model(n_hidden_layers, n_neurons, dropout_rate, activation):
    if activation == 'relu':
        kernel_init = 'he_uniform'
        DropoutType = Dropout
        apply_batch_norm = True
    elif activation == 'selu':
        kernel_init = 'lecun_normal'
        DropoutType = AlphaDropout
        apply_batch_norm = False
    else:
        raise RuntimeError('Activation "{}" is not supported'.format(activation))
    
    model_name = "{}L{}N{}D_{}".format(n_hidden_layers, n_neurons, int(dropout_rate*100), activation)
    input_layer = Input(name="main_input", shape=input_shape)
    
    prev_layer = input_layer
    for n in range(n_hidden_layers):
        dense_layer = Dense(n_neurons, name="dense_%d" % n, kernel_initializer=kernel_init)(prev_layer)
        layer = dense_layer
        if apply_batch_norm:
            layer = BatchNormalization(name="batch_norm_%d" % n)(dense_layer)
        activation_layer = Activation(activation, name="activation_%d" % n)(layer)
        prev_layer = DropoutType(dropout_rate, name="dropout_%d" % n)(activation_layer)
    output_layer = Dense(n_outputs, name="dense_%d" % n_hidden_layers)(prev_layer)
    softmax_output = Activation("softmax", name="main_output")(output_layer)

    weight_input = Input(name="weight_input", shape=(3,))
    model = Model([input_layer, weight_input], softmax_output, name="deepTauId")
    return model, model_name

In [10]:
def compile_model(model, learning_rate):
    opt = keras.optimizers.Adam(lr=learning_rate)
    #opt = keras.optimizers.Nadam(lr=learning_rate)
    #opt = keras.optimizers.SGD(lr=learning_rate, momentum=0.9, nesterov=True)
    #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
    
    weight_input = model.inputs[1]
    tau_crossentropy = functools.partial(TauLosses.tau_crossentropy, weights=weight_input)
    Le = functools.partial(TauLosses.Le, weights=weight_input)
    Lmu = functools.partial(TauLosses.Lmu, weights=weight_input)
    Ljet = functools.partial(TauLosses.Ljet, weights=weight_input)
    sLe = functools.partial(TauLosses.sLe, weights=weight_input)
    sLmu = functools.partial(TauLosses.sLmu, weights=weight_input)
    sLjet = functools.partial(TauLosses.sLjet, weights=weight_input)
    
    functools.update_wrapper(tau_crossentropy, TauLosses.tau_crossentropy)
    functools.update_wrapper(Le, TauLosses.Le)
    functools.update_wrapper(Lmu, TauLosses.Lmu)
    functools.update_wrapper(Ljet, TauLosses.Ljet)
    functools.update_wrapper(sLe, TauLosses.sLe)
    functools.update_wrapper(sLmu, TauLosses.sLmu)
    functools.update_wrapper(sLjet, TauLosses.sLjet)
    
    model.compile(loss=tau_crossentropy, optimizer=opt, metrics=["accuracy", Le, Lmu, Ljet, sLe, sLmu, sLjet])

In [11]:
model, model_name = create_model(20, 1024, 0.5, 'relu')
#model, model_name = create_model(20, 1024, 0.5, 'selu')

model.load_weights("20L1024N50D_relu_0723_s1_final.hdf5", by_name=True)

compile_model(model, 1e-3)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 181)               0         
_________________________________________________________________
dense_0 (Dense)              (None, 1024)              186368    
_________________________________________________________________
batch_norm_0 (BatchNormaliza (None, 1024)              4096      
_________________________________________________________________
activation_0 (Activation)    (None, 1024)              0         
_________________________________________________________________
dropout_0 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
batch_norm_1 (BatchNormaliza (None, 1024)              4096      
__________

_________________________________________________________________
dense_15 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
batch_norm_15 (BatchNormaliz (None, 1024)              4096      
_________________________________________________________________
activation_15 (Activation)   (None, 1024)              0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
batch_norm_16 (BatchNormaliz (None, 1024)              4096      
_________________________________________________________________
activation_16 (Activation)   (None, 1024)              0         
_________________________________________________________________
dropout_16

In [12]:
def close_file(f_name):
    file_objs = [ obj for obj in gc.get_objects() if ("TextIOWrapper" in str(type(obj))) and (obj.name == f_name)]
    for obj in file_objs:
        obj.close()

In [13]:
def run_training(train_suffix, model_name, epoch, n_pass):
    n_train = int(6e7)
    n_test = X.shape[0] - n_train
    n_per_step = int(1e7)

    train_name = '%s_%s' % (model_name, train_suffix)
    
    weightManager.SetHistFileName('%s_weight_upd.hdf5' % train_name)
    
    cb_acc = ModelCheckpoint("%s_acc.hdf5" % train_name, monitor="val_acc", save_best_only=True,
                             save_weights_only=False, mode="max", verbose=1)
    
    cb_losses = []
    for loss_name in ["loss", "Le", "Lmu", "Ljet"]:
        cb_losses.append(ModelCheckpoint("%s_%s.hdf5" % (train_name, loss_name), monitor="val_%s" % loss_name,
                                         save_best_only=True, save_weights_only=False, mode="min", verbose=1))

    
    log_name = "%s.log" % train_name
    if os.path.isfile(log_name):
        close_file(log_name)
        os.remove(log_name)
    csv_log = CSVLogger(log_name, append=True)

    for pass_id in range(0, n_pass):
        print("Starting pass %s" % pass_id)
        for pos in range(0, n_train, n_per_step):
            if epoch != 0:
                print("Updating weights...")
                weightManager.UpdateWeights(model, epoch, X, pos, n_per_step,
                                            [('jet', 0.6), ('e', 0.6), ('mu', 0.9)])
            
            val_data = ([X[n_train:n_train+n_test, :], weightManager.GetWeights(n_train, n_train+n_test)],
                        Y[n_train:n_train+n_test, :])
            pbar = TQDMNotebookCallback(leave_outer=False, show_outer=False, leave_inner = True)
            callbacks = [pbar, csv_log, cb_acc, *cb_losses]
            fit_hist = model.fit([X[pos:pos+n_per_step, :], weightManager.GetWeights(pos, pos+n_per_step)],
                                 Y[pos:pos+n_per_step, :], callbacks=callbacks, validation_data=val_data, batch_size=10000,
                                 epochs=epoch+1, initial_epoch=epoch, verbose=0)
            epoch += 1
        if pass_id + 1 != n_pass:
            weightManager.SaveWeights('%s_last_pass_weights.hdf5' % train_name)
            model.save("%s_last_pass.hdf5" % train_name)

    weightManager.SaveWeights('%s_final_weights.hdf5' % train_name)
    model.save("%s_final.hdf5" % train_name)

In [14]:
#run_training('0723_s1', model_name, 0, 50)
run_training('0723_s1p2', model_name, 300, 50)

Starting pass 0
Updating weights...
	predictions has been calculated.
tau_vs_jet: bins changed = 217, average sf = 0.9986306864360228
tau_vs_e: bins changed = 238, average sf = 1.0021491739952713
tau_vs_mu: bins changed = 89, average sf = 0.9903144651041345




Epoch 00301: val_acc improved from -inf to 0.77558, saving model to 20L1024N50D_relu_0723_s1p2_acc.hdf5

Epoch 00301: val_loss improved from inf to 2.05340, saving model to 20L1024N50D_relu_0723_s1p2_loss.hdf5

Epoch 00301: val_Le improved from inf to 0.61399, saving model to 20L1024N50D_relu_0723_s1p2_Le.hdf5

Epoch 00301: val_Lmu improved from inf to 0.27789, saving model to 20L1024N50D_relu_0723_s1p2_Lmu.hdf5

Epoch 00301: val_Ljet improved from inf to 0.17673, saving model to 20L1024N50D_relu_0723_s1p2_Ljet.hdf5
Updating weights...
	predictions has been calculated.
tau_vs_jet: bins changed = 252, average sf = 1.0057235305876044
tau_vs_e: bins changed = 202, average sf = 1.0004021175785789
tau_vs_mu: bins changed = 217, average sf = 0.9911142179136636




Epoch 00302: val_acc improved from 0.77558 to 0.93695, saving model to 20L1024N50D_relu_0723_s1p2_acc.hdf5

Epoch 00302: val_loss improved from 2.05340 to 0.96203, saving model to 20L1024N50D_relu_0723_s1p2_loss.hdf5

Epoch 00302: val_Le improved from 0.61399 to 0.18444, saving model to 20L1024N50D_relu_0723_s1p2_Le.hdf5

Epoch 00302: val_Lmu improved from 0.27789 to 0.04108, saving model to 20L1024N50D_relu_0723_s1p2_Lmu.hdf5

Epoch 00302: val_Ljet improved from 0.17673 to 0.13908, saving model to 20L1024N50D_relu_0723_s1p2_Ljet.hdf5
Updating weights...
	predictions has been calculated.
tau_vs_jet: bins changed = 240, average sf = 1.002022711997095
tau_vs_e: bins changed = 225, average sf = 1.0012018024335887
tau_vs_mu: bins changed = 29, average sf = 1.0792889957331118




Epoch 00303: val_acc did not improve from 0.93695

Epoch 00303: val_loss did not improve from 0.96203

Epoch 00303: val_Le did not improve from 0.18444

Epoch 00303: val_Lmu did not improve from 0.04108

Epoch 00303: val_Ljet did not improve from 0.13908
Updating weights...
	predictions has been calculated.
tau_vs_jet: bins changed = 172, average sf = 1.0036167498096216
tau_vs_e: bins changed = 280, average sf = 1.0047197143377447
tau_vs_mu: bins changed = 64, average sf = 0.9760175384698011




Epoch 00304: val_acc improved from 0.93695 to 0.93936, saving model to 20L1024N50D_relu_0723_s1p2_acc.hdf5

Epoch 00304: val_loss did not improve from 0.96203

Epoch 00304: val_Le did not improve from 0.18444

Epoch 00304: val_Lmu improved from 0.04108 to 0.04032, saving model to 20L1024N50D_relu_0723_s1p2_Lmu.hdf5

Epoch 00304: val_Ljet did not improve from 0.13908
Updating weights...
	predictions has been calculated.
tau_vs_jet: bins changed = 232, average sf = 0.9997457539830789
tau_vs_e: bins changed = 228, average sf = 1.0014948550103646
tau_vs_mu: bins changed = 56, average sf = 1.0276444601954473


...



Epoch 00340: val_acc did not improve from 0.95059

Epoch 00340: val_loss did not improve from 0.91564

Epoch 00340: val_Le did not improve from 0.18430

Epoch 00340: val_Lmu did not improve from 0.03137

Epoch 00340: val_Ljet did not improve from 0.13106
Updating weights...
	predictions has been calculated.
tau_vs_jet: bins changed = 191, average sf = 1.0003428453830263
tau_vs_e: bins changed = 207, average sf = 0.9993100182305839
tau_vs_mu: bins changed = 105, average sf = 0.9881266203589182




Epoch 00341: val_acc did not improve from 0.95059

Epoch 00341: val_loss did not improve from 0.91564

Epoch 00341: val_Le did not improve from 0.18430

Epoch 00341: val_Lmu did not improve from 0.03137

Epoch 00341: val_Ljet did not improve from 0.13106
Updating weights...
	predictions has been calculated.
tau_vs_jet: bins changed = 214, average sf = 1.002291333196693
tau_vs_e: bins changed = 158, average sf = 1.0005905021693575
tau_vs_mu: bins changed = 45, average sf = 0.987650027049641




Epoch 00342: val_acc did not improve from 0.95059

Epoch 00342: val_loss did not improve from 0.91564

Epoch 00342: val_Le did not improve from 0.18430

Epoch 00342: val_Lmu did not improve from 0.03137

Epoch 00342: val_Ljet did not improve from 0.13106
Starting pass 7
Updating weights...
	predictions has been calculated.
tau_vs_jet: bins changed = 179, average sf = 1.002239962050933
tau_vs_e: bins changed = 188, average sf = 1.0020791298204188
tau_vs_mu: bins changed = 129, average sf = 0.9746066064354635




Epoch 00343: val_acc did not improve from 0.95059

Epoch 00343: val_loss did not improve from 0.91564

Epoch 00343: val_Le did not improve from 0.18430

Epoch 00343: val_Lmu did not improve from 0.03137

Epoch 00343: val_Ljet did not improve from 0.13106
Updating weights...


KeyboardInterrupt: 